In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
link = 'https://drive.google.com/file/d/1AOr7OB3OvauKrJ-I7rf6SiGui5YcKn6-/view?usp=sharing'

# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Real_Combine.csv') 
 
df = pd.read_csv('Real_Combine.csv')
print(df.shape)
df.head()

(1093, 9)


,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [4]:
from tensorflow import keras
from keras import layers
!pip install -q -U keras-tuner
from keras_tuner.tuners import RandomSearch

     |████████████████████████████████| 133 kB 5.1 MB/s 


In [5]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

Hyperparameters:
1. How many number of hidden layers we should have?
2. How many numbers of neuron we should have in hidden layers?
3. What will be the learning rate?

In [6]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers', 2, 20)):
    model.add(layers.Dense(units = hp.Int('units_' + str(i),
                                        min_value = 32,
                                        max_value = 512,
                                        step = 32),
                           activation ='relu'))
  
  model.add(layers.Dense(1, activation='linear'))
  model.compile(
      optimizer=keras.optimizers.Adam(
          hp.Choice('learning_rate', [1e-2,1e-3,1e-4])),
          loss = 'mean_absolute_error',
          metrics = ['mean_absolute_error'])
  return model

In [7]:
tuner = RandomSearch(
    build_model,
    objective = 'val_mean_absolute_error',
    max_trials = 5,
    executions_per_trial = 3,
    directory='project',
    project_name='Air Quality Index'
)

In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [10]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 11s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 00m 54s
INFO:tensorflow:Oracle triggered exit


In [11]:
tuner.results_summary()

Results summary
Results in project/Air Quality Index
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 8
units_0: 256
units_1: 320
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 11
units_0: 352
units_1: 384
learning_rate: 0.001
units_2: 96
units_3: 512
units_4: 64
units_5: 32
units_6: 352
units_7: 448
units_8: 32
units_9: 32
units_10: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 3
units_0: 320
units_1: 384
learning_rate: 0.001
units_2: 320
units_3: 192
units_4: 480
units_5: 64
units_6: 224
units_7: 352
units_8: 352
units_9: 160
units_10: 128
Score: nan
Trial summary
Hyperparameters:
num_layers: 12
units_0: 256
units_1: 384
learning_rate: 0.0001
units_2: 352
units_3: 160
units_4: 256
units_5: 352
units_6: 160
units_7: 160
units_8: 352
units_9: 96
units_10: 384
units_11: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 14
units_0: 352
units_1: 224
learnin